In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import psycopg2
from sklearn.cluster import KMeans

In [ ]:
def re_group(place_df,day):
  result = [ [] for _ in range(day) ]
  kmeans = KMeans(n_clusters=day,random_state=0)
  kmeans = kmeans.fit(place_df[['latitude','longitude']])
  place_df['cluster'] = kmeans.labels_
  for i in place_df.values:
    id,latitude,longitude,group = i
    result[group].append(id)
  return result

##course 불러오기

In [7]:
course = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/08.09_course.csv",index_col=0)
course.head(2)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location
0,1,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,"[602, 10, 58, 29]",1.250000,2.000000,1.250000,1.250000,2.750000,1.000000,1.750000,0.250000,3,4,2.0,1.0,0.0,1.0,Mixed
1,2,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,"[19, 37, 10, 602, 60, 58, 29]",1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2.285714,0.428571,3,7,2.0,2.0,0.0,3.0,Mixed


In [11]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/07.31_attraction_db.csv",index_col=0)
attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,...,2,2,1,1,3,1,0,0,8,"자연경관,커플,봄,흐림,걷기/등산,도보,친구"
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,...,3,3,3,1,3,0,3,0,7,"자연경관,맑음,아이,포토스팟"


##cluster 별로 특징 파악하기

In [9]:
course.groupby('cluster').mean().sort_values("nature")

,id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,length,east,west,south,north
cluster,,,,,,,,,,,,,,
2,1831.107692,0.911183,1.123805,1.062423,1.154620,1.203449,1.035265,2.415785,1.301892,6.507692,1.046154,2.469231,1.223077,1.769231
9,1983.779006,1.427902,1.694756,1.053841,1.263781,1.569757,1.014708,2.089635,0.463624,7.287293,1.121547,2.801105,1.513812,1.850829
11,1895.508696,1.541889,2.091693,1.342169,1.489801,1.327160,1.565568,2.560128,0.389292,8.078261,1.695652,2.508696,1.739130,2.134783
10,1978.366667,1.582162,2.008507,1.389495,1.224670,1.714438,0.840804,2.526225,0.988655,9.453333,2.270000,2.816667,2.020000,2.346667
3,1884.518519,1.834318,2.300192,1.444049,1.409030,2.130678,0.976016,2.515802,0.287909,9.632099,2.923457,3.049383,1.306173,2.353086
0,1744.540936,2.060918,2.427954,1.659310,1.166316,2.022475,0.609041,2.599315,0.733124,11.324561,3.719298,2.637427,2.403509,2.564327
1,1998.137255,2.076963,2.432548,1.248304,1.431477,1.714696,0.856287,2.094806,0.363598,9.588235,1.708683,2.924370,2.473389,2.481793
6,2002.212042,2.122126,2.630481,1.624091,1.430735,1.737183,1.254982,2.632921,0.268290,10.455497,3.081152,2.649215,2.515707,2.209424
5,1751.832808,2.226208,2.793137,1.583189,2.081434,2.136172,1.223631,2.672131,0.166690,8.429022,2.892744,2.293375,1.069401,2.173502


In [137]:
sample = course.loc[course['cluster'] == 8].sample()
sample

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location
3697,3698,제주도!,"[244, 8, 59, 28, 24, 419, 2, 26, 4, 6, 9]",2.545455,2.909091,1.909091,1.272727,2.545455,0.363636,2.545455,0.181818,8,11,5.0,4.0,0.0,2.0,Mixed


In [138]:
check = id_to_title(sample,attraction)
check

['용담/용두암 해안도로',
 '협재해수욕장',
 '애월해안도로',
 '한림공원',
 '용머리해안',
 '사계해안도로',
 '우도(해양도립공원)',
 '용눈이오름',
 '비자림',
 '만장굴',
 '섭지코지']

cluster 별 특징 title
0 : 쉿! 같이 걷자 제주
1 : 제주에선, 잠시 쉬어가도 괜찮아.
2 : 실내에서 즐기는 제주 인문학 탐방
3 : 20대라면, 사진 찍기 좋은 제주 감성여행
4 : 부모님과 함께라면, 더 좋은 제주
5 : 맑고 푸르른 제주의 쪽빛 바다
6 : 몸으로 체험하는 제주의 자연
7 : 화산이 빚어낸 제주의 고요한 녹빛
8 : 시간이 빚어낸 제주의 푸르른 경관
9 : 실내외를 아우르는 제주여행
10 : 제주, 다양하게 즐기기
11 : 신나게 즐겨보는 액티브 인 제주



In [ ]:
course.groupby('length').describe()

In [ ]:
aa

###코스 구성 목적지 Title 반환 메소드 (df를 csv파일로 불러왔을때 버전)

In [51]:
def id_to_title(course_df,attraction_df):
  places= course_df['places'].values[0]
  places = places[1:-1].split(',')
  places = list(map(int,places))
  print(places)
  title_list = []
  for i in places:
    now = attraction_df.loc[attraction_df['id'] == i,'title'].values[0]
    title_list.append(now)
  return title_list

In [40]:
test =  id_to_title(course.iloc[0],attraction)
test

Now :  수목원테마공원(야시장길&LED공원)
Now :  카멜리아힐
Now :  성읍민속마을
Now :  아쿠아플라넷 제주


['수목원테마공원(야시장길&LED공원)', '카멜리아힐', '성읍민속마을', '아쿠아플라넷 제주']

In [22]:
attraction.loc[attraction['id']==100,'title'].values

array(['월정리 카약체험장'], dtype=object)

##일자별로 나누기

In [152]:
import json

In [153]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [147]:
attraction.loc[attraction['id'] == 1 ]

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,...,2,2,1,1,3,1,0,0,8,"자연경관,커플,봄,흐림,걷기/등산,도보,친구"


###코스에서 places에서 좌표 따와서 dataFrame으로 만들어주기

In [172]:
db = load_db()
cursor = db.cursor()

In [173]:
def load_places_location(place_list):
  global db
  global cursor
  #place_list = [1,2,3,4,5,6,7,8,9]
  
  place_list = place_list[1:-1].split(',')
  place_list = tuple(map(int,place_list))
  place_list = str(place_list)

  
  sql = f"select place_id, latitude, longitude from places where place_id in {place_list}"
  cursor.execute(sql)
  result = cursor.fetchall()
  result = pd.DataFrame(result)
  result.columns = ['id','latitude','longitude']
  return result

In [175]:
sample_course = course.sample(10)

In [186]:
course['optimize'] = course['places'].apply(load_places_location)

In [188]:
course['optimize'].values[0]

,id,latitude,longitude
0,10,33.2901402,126.3683652
1,29,33.4326445,126.9279667
2,58,33.3845107,126.8003768
3,602,33.4706602,126.4879244


###좌표 정보가 있는 dataFrame을 가지고 최적화 해주고 저장해주기

In [195]:
def re_group(place_df):
  place_df.drop_duplicates(['id'],inplace=True)
  day = len(place_df)//3
  if len(place_df)<3:
    return []
    
  result = [ [] for _ in range(day) ]
  
  kmeans = KMeans(n_clusters=day,random_state=0)
  kmeans = kmeans.fit(place_df[['latitude','longitude']])
  place_df['cluster'] = kmeans.labels_
  for i in place_df.values:
    id,latitude,longitude,group = i
    result[group].append(id)
  return result

In [196]:
course['optimized_route'] = course['optimize'].apply(re_group)

In [199]:
checking = course.sample()
checking

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location,optimize,optimized_route
2157,2158,10월 제주 3일,"[152, 236, 174, 244, 128, 192, 554, 361, 8, 26...",2.142857,2.357143,1.428571,1.285714,2.071429,0.285714,2.642857,1.071429,0,14,4.0,5.0,2.0,3.0,Mixed,id latitude longitude cluster 0 ...,"[[8, 24, 152, 174, 236, 390], [26, 145, 192, 3..."


In [204]:
checking['optimized_route'].values[0]

[[8, 24, 152, 174, 236, 390], [26, 145, 192, 361, 554], [128, 244], [488]]

In [205]:
course['day'] = course['optimized_route'].apply(lambda x: len(x))

In [208]:
#db에에 적용할 코스
course_to_applyDb = course.loc[course['day']<8]

###자고일어나서 코스 다시 로드한다음에 (pickle) 범철이한테 물어본 뒤에 아래 for loop 수행해야함!

In [233]:
for i in range(3743):
  now = course_to_applyDb.iloc[2446]
  now_path = now['optimized_route']
  now_cluster = now['cluster']
  now_location_info = list(now[['east','west','south','north']])
  max_idx = now_location_info.argmax()
  max_ratio = now_location_info[max_idx] / now_location_info.sum()
  region = []
  if max_ratio > 0.5:
    if max_idx == 0:
      region.append("제주 동부")
    elif max_idx==1:
      region.append("제주 서부")
    elif max_idx==2:
      region.append("제주 남부")
    else:
      region.append("제주 북부")
  else:
    if now_location_info[0] > 0 :
      region.append("제주 동부")
    if now_location_info[1] > 0 :
      region.append("제주 서부")
    if now_location_info[2] > 0 :
      region.append("제주 남부")
    if now_location_info[3] > 0 :
      region.append("제주 북부")



  print('cluster : ',now_cluster  ,  "location : ",now_location_info)
  travel_note_id = i+1
  member_id = 1
  title = "temp"
  #title = cluster별로 달랐던거 적용하기!!
  #region = now_location_info 참조하기!!
  #insert into travel_note(member_id, title, day_start,day_end,adult,region)
  
  sql = "insert into travel_note(member_id, title, day_start,day_end,adult,region) values (%s %s %s %s %s %s)",(1,title,"start","end",region)
  cursor.execute(sql)
  db.commit()
  for idx,path in enumerate(now_path):
    #insert into course(course_id, travel_note_id, day, places) VALUES(%s, %s, %s, %s )",(10000+ idx+1, travel_note_id, idx+1 path)
    print(path)
    #course_id는 idx처럼 바뀌고
    #travel_note_id는 
  break


cluster :  4 location :  [3.0, 3.0, 3.0, 1.0]
[2, 6, 9]
[5, 15, 16, 37]
[8, 28, 38]


In [230]:
course_to_applyDb.groupby('day').count()

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,culture,cluster,length,east,west,south,north,main_location,optimize,optimized_route
day,,,,,,,,,,,,,,,,,,,
0,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
1,914,914,914,914,914,914,914,914,914,914,914,914,914,914,914,914,914,914,914
2,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099
3,799,799,799,799,799,799,799,799,799,799,799,799,799,799,799,799,799,799,799
4,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466
5,246,246,246,246,246,246,246,246,246,246,246,246,246,246,246,246,246,246,246
6,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134
7,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67


###일단 이것저것 적용한 코스 파일 저장해놓자 (dataframe있으니까 pickle로)

In [210]:
course_to_applyDb.to_pickle("/content/drive/MyDrive/yeoreodigm/data_files/08.14_course.pickle")

###pickle파일 저장한거 확인해보기

In [234]:
pickle_course = pd.read_pickle("/content/drive/MyDrive/yeoreodigm/data_files/08.14_course.pickle")

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location,optimize,optimized_route
1964,1964,0312제주0314,"[3, 52, 2, 134]",3.0,3.0,2.25,1.75,2.25,0.5,1,0.5,8,4,4.0,0.0,0.0,0.0,East,id latitude longitude cluster 0 ...,"[[2, 3, 52, 134]]"


In [241]:
sample_pickle = pickle_course.sample()
sample_pickle['optimize'].values[0]

,id,latitude,longitude,cluster
0,29,33.4326445,126.9279667,0
1,59,33.4670218,126.3378989,0
2,80,33.4521111,126.4884830,0


In [ ]:
#place_list로는 df에서 코스를 불러와서 places를 전달해주면 됨. (string이니까 list로 변환 해줄것.)
#place_list를 바탕으로 id, 위도, 경도의 df형식으로 반환해주면 해당 df를 인자로 받아서 날짜별로 쪼개줌.
#df에서 날짜별로 쪼갠걸 바탕으로 index와 list를 같이 course에 넣어주면 됨.(for loop)

In [ ]:
places = course['places'].values[0]
places = places[1:-1].split(',')
places = list(map(int,places))

In [ ]:
for day in range(1,8):
  min = 3*day
  max = 3*(day+1)-1
  #min~max 까지 n일 일정으로 클러스터링 해야함
  now = course.loc[(course['length']>=min) & (course['length']<=min)]
  now['optimized_path'] = now['']



In [167]:
tmp = course.loc[course['length']==3]
for i,val in enumerate(tmp.values):
  print(val)
  print(type(val))
  if i == 3:
    break


[4 '[청주출발]"모두 알뜰 패키지" 제주 관광호텔 2박3일 비밀의 그림자+ 웰빙 족욕' '[37, 202, 58]' 2.0 3.0
 1.6666666666666667 1.6666666666666667 1.6666666666666667 1.0 3.0
 0.6666666666666666 6 3 1.0 0.0 0.0 2.0 'North']
<class 'numpy.ndarray'>
[9 '[부산출발][코시롱한나들이]♥감귤타르트증정♥제주 아쿠아플라넷 아스타호텔 3일' '[78, 60, 29]'
 0.6666666666666666 1.0 1.3333333333333333 1.0 2.0 1.3333333333333333
 2.333333333333333 1.3333333333333333 2 3 1.0 2.0 0.0 0.0 'West']
<class 'numpy.ndarray'>
[10 '[부산출발][노팁노옵션]♥흑돼지육포증정♥제주 잠수함여행 아스타호텔 3일' '[78, 118, 9]' 2.0
 1.6666666666666667 1.6666666666666667 1.6666666666666667
 1.3333333333333333 2.0 2.6666666666666665 1.0 11 3 2.0 1.0 0.0 0.0 'East']
<class 'numpy.ndarray'>
[11 '[부산출발][섬속의섬]♥감귤타르트증정♥우도투어+비밀의그림자공연 제주 아스타호텔 3일' '[78, 10, 2]'
 1.6666666666666667 1.6666666666666667 1.6666666666666667 1.0
 2.333333333333333 1.3333333333333333 2.333333333333333 1.0 10 3 1.0 2.0
 0.0 0.0 'West']
<class 'numpy.ndarray'>
